In [1]:
import sys; sys.path.insert(0, '..')

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from joblib import dump, load
from matplotlib.colors import ListedColormap
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer
from sklearn.model_selection import (GridSearchCV, cross_validate,
                                     train_test_split)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from tqdm import tqdm

from scripts.get_data import ceramiche_DB

In [3]:
classifiers = {
    "Neural net": (
                MLPClassifier(),
                {
                    'hidden_layer_sizes': list(range(2,30)),
                    'activation': ['identity', 'logistic', 'tanh', 'relu'],
                }
            ),


    "Nearest Neighbors": (
                    KNeighborsClassifier(),
                    {'n_neighbors': list(range(1,11)), 'weights': ['uniform', 'distance']}
                ),
    "SVC": (
                    SVC(),
                    [{
                        'kernel': ['linear', 'rbf', 'sigmoid'],
                        'C': [x/10 for x in range(1, 11)],
                    },
                    {
                        'kernel': ['poly'],
                        'degree': list(range(2,6)),
                        'C': [x/10 for x in range(1, 11)],
                    }]
                ),
    "Decision Tree": (
                    DecisionTreeClassifier(),
                    {
                        'criterion': ['gini', 'entropy'], 
                        'min_samples_split':list(range(2,11)),
                        'min_samples_leaf': list(range(1,6)),
                    }
                ),
    "Random Forest": (
                    RandomForestClassifier(n_jobs=-1),
                    {
                        'n_estimators': list(range(1, 101, 10)), 
                        'criterion': ['gini', 'entropy'], 
                        'min_samples_split':list(range(2, 11)),
                        'min_samples_leaf': list(range(1, 6)),
                        'oob_score': [True, False],
                    }
                ),
    "Naive Bayes": (
                    GaussianNB(),
                    {}
                ),

    # "Gaussian Process": GaussianProcessClassifier(1.0 * RBF(1.0)),
    # "AdaBoost": AdaBoostClassifier(),
    # "QDA": QuadraticDiscriminantAnalysis(),
}

In [4]:
def sensibility(y_true, y_predict):
    c_matrix = confusion_matrix(y_true, y_predict)
    return c_matrix[1][1]/sum(c_matrix[1])


def specificity(y_true, y_predict):
    c_matrix = confusion_matrix(y_true, y_predict)
    return c_matrix[0][0]/sum(c_matrix[0])


scores = {
    'sensibility' : make_scorer(sensibility, greater_is_better=True),
    'specificity' : make_scorer(specificity, greater_is_better=True),
    'accuracy' : make_scorer(accuracy_score, greater_is_better=True)
}

In [5]:
X, y = ceramiche_DB()
X_train, X_test, y_train, y_test = train_test_split(X, y)

..\dati_ceramiche_classi.xlsx


In [9]:
#Se voi eseguire di nuovo la computazione invcece che caricare quella cachata cambia questa variabile
NEW_COMPUTATION = False

if NEW_COMPUTATION:
    result = {}

    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        for name, (clf, params) in tqdm(classifiers.items()):
            grid = GridSearchCV(clf, params, scoring='accuracy', cv = 5, n_jobs = -1)
            s = cross_validate(grid, X, y, cv=5, scoring = scores, return_estimator=True)
            result[name] = s
else:
    result = load('../scripts/supervised_dump.joblib')


In [7]:
rows = ['sensibility', 'specificity', 'accuracy']
columns = ['mean', 'std']

row_format ="{:>20}" * (len(columns) + 1)
col_format ="{:>20}" * (len(columns) + 1)

for n, r in result.items():
    print(n.upper())

    data = [(r["test_"+x].mean(), r["test_"+x].std()) for x in rows]

    print(col_format.format("", *columns))
    for name, row in zip(rows, data):
        print(row_format.format(name, *row))

    print()
    print()


NEURAL NET
                                    mean                 std
         sensibility  0.9286764705882353 0.11399667633107935
         specificity  0.7999999999999999 0.16329931618554522
            accuracy  0.8946969696969698 0.06794571916599595


NEAREST NEIGHBORS
                                    mean                 std
         sensibility  0.9522058823529411 0.06874115230894264
         specificity  0.9333333333333333 0.13333333333333336
            accuracy  0.9477272727272726 0.04958505506652598


SVC
                                    mean                 std
         sensibility  0.9404411764705882 0.09114088489615395
         specificity  0.8666666666666666 0.16329931618554522
            accuracy  0.9212121212121213 0.05538080873483959


DECISION TREE
                                    mean                 std
         sensibility  0.9036764705882353 0.12269501108009499
         specificity  0.9333333333333333 0.13333333333333336
            accuracy  0.91287878

In [8]:
# dump(result, '../scripts/supervised_dump.joblib')

['../scripts/supervised_dump.joblib']